In [4]:
cd drive/MyDrive/Classification/

[Errno 2] No such file or directory: 'drive/MyDrive/Classification/'
/content/drive/MyDrive/Classification


In [5]:
import config
import tensorflow as tf
from models.resnet import resnet_18, resnet_34, resnet_50, resnet_101, resnet_152
from tensorflow.keras import layers

import config
import pathlib
from config import image_height, image_width, channels


def load_and_preprocess_image(img_path):
    # read pictures
    img_raw = tf.io.read_file(img_path)
    # decode pictures
    img_tensor = tf.image.decode_jpeg(img_raw, channels=channels)
    # resize
    img_tensor = tf.image.resize(img_tensor, [image_height, image_width])
    img_tensor = tf.cast(img_tensor, tf.float32)
    # normalization
    img = img_tensor / 255.0
    return img, img_path


def get_images_and_labels(data_root_dir):
    # get all images' paths (format: string)
    data_root = pathlib.Path(data_root_dir)
    all_image_path = [str(path) for path in list(data_root.glob('*/*'))]
    # get labels' names
    label_names = sorted(item.name for item in data_root.glob('*/'))
    # dict: {label : index}
    label_to_index = dict((label, index) for index, label in enumerate(label_names))
    # get all images' labels
    all_image_label = [label_to_index[pathlib.Path(single_image_path).parent.name] for single_image_path in all_image_path]

    #print("label:")
    #print(all_image_label)
    return all_image_path, all_image_label


def get_dataset(dataset_root_dir):
    all_image_path, all_image_label = get_images_and_labels(data_root_dir=dataset_root_dir)
    # print("image_path: {}".format(all_image_path[:]))
    # print("image_label: {}".format(all_image_label[:]))
    # load the dataset and preprocess images
    image_dataset = tf.data.Dataset.from_tensor_slices(all_image_path).map(load_and_preprocess_image)
    label_dataset = tf.data.Dataset.from_tensor_slices(all_image_label)
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    image_count = len(all_image_path)

    return dataset, image_count


def generate_datasets():
    train_dataset, train_count = get_dataset(dataset_root_dir=config.train_dir)
    print("train_count: %d", train_count)
    valid_dataset, valid_count = get_dataset(dataset_root_dir=config.valid_dir)
    print("valid_count: %d", valid_count)
    test_dataset, test_count = get_dataset(dataset_root_dir=config.test_dir)
    print("test_count: %d", test_count)


    # read the original_dataset in the form of batch
    train_dataset = train_dataset.shuffle(buffer_size=train_count).batch(batch_size=config.BATCH_SIZE)
    valid_dataset = valid_dataset.batch(batch_size=config.BATCH_SIZE)
    test_dataset = test_dataset.batch(batch_size=config.BATCH_SIZE)

    return train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count



def get_model(model_name):
    
    # model=torchvision.models.resnet18(pretrained=True)
    # num_features=model.fc.in_features
    # model.fc=tf.nn.Linear(num_features,config.NUM_CLASSES)
    model = resnet_18()
    if model_name == "resnet50":
        model = resnet_50()
    if model_name == "resnet34":
        model = resnet_34()
    if model_name == "resnet101":
        model = resnet_101()
    if model_name == "resnet152":
        model = resnet_152()
    # model.build(input_shape=(None, config.image_height, config.image_width, config.channels))
    # model.summary()
    return model

# GPU settings
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets()

train_count: %d 60000
valid_count: %d 18000
test_count: %d 18000


In [28]:
resnet18 = tf.keras.models.load_model('../Visualization/saved_model/keras/resnet18')
resnet50 = tf.keras.models.load_model('../Visualization/saved_model/keras/resnet_50')
resnet50_pre = tf.keras.models.load_model('../Visualization/saved_model/keras/resnet50_pre')

In [30]:
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np

def predict_proba(model, dataset):
  model.compile()
  y_pred = []
  y_truth = []
  for (images, paths), labels in tqdm(dataset):
    pred = model(images, training=False)
    probs = layers.Softmax(axis=-1)(pred).numpy()
    features = []
    y_pred.extend(probs)
    y_truth.extend(labels.numpy())
  return y_pred, y_truth

In [26]:
probs, labels = predict_proba(resnet18, test_dataset)
np.save('resnet18_probs_test', probs)
np.save('resnet18_labels_test', labels)

  0%|          | 0/2250 [00:01<?, ?it/s]


AttributeError: ignored

In [ ]:
probs, labels = predict_proba(resnet50, test_dataset)
np.save('resnet50_probs_test', probs)
np.save('resnet50_labels_test', labels)

100%|██████████| 2250/2250 [1:46:13<00:00,  2.83s/it]


In [31]:
probs, labels = predict_proba(resnet50_pre, test_dataset)
np.save('resnet50pre_probs_test', probs)
np.save('resnet50pre_labels_test', labels)

100%|██████████| 2250/2250 [1:51:21<00:00,  2.97s/it]
